In [11]:
import pandas as pd
import numpy as np
import ast

# CSV 파일 불러오기
df_inheritance = pd.read_csv('df_inheritance_kobertmodel.csv')

# 다시 리스트를 NumPy 배열로 변환
df_inheritance['판례내용이유임베딩'] = df_inheritance['판례내용이유임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)
df_inheritance['키워드임베딩'] = df_inheritance['키워드임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)
df_inheritance['판결요지임베딩'] = df_inheritance['판결요지임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)
df_inheritance['판시사항임베딩'] = df_inheritance['판시사항임베딩'].apply(lambda x: np.array(ast.literal_eval(x)) if pd.notnull(x) else None)


In [12]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')


# 문장 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [21]:
# 각 행의 키워드 문자열을 쉼표로 분리하고 리스트 형태로 저장
df_inheritance['키워드'] = df_inheritance['키워드'].apply(lambda x: x.split(', '))

In [22]:
# 사용자 키워드에 대한 임베딩을 찾기 위한 함수
def get_keyword_embedding(user_keyword, df):
    # 키워드가 포함된 행을 필터링
    matching_rows = df[df['키워드'].apply(lambda x: user_keyword in x if isinstance(x, list) else False)]
    
    if not matching_rows.empty:
        # 첫 번째 매칭된 키워드의 임베딩을 반환
        return matching_rows['키워드임베딩'].iloc[0]
    else:
        return None  # 매칭되는 키워드가 없으면 None 반환

In [35]:
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드

# 각각 임베딩 생성(1 매칭키워드임베딩찾기, 2 Tokenizer사용)
user_keyword_embedding_1 = get_keyword_embedding(user_keyword, df_inheritance)
user_keyword_embedding_2 = encode_sentence(user_keyword)

# 임베딩 값 출력
print("get_keyword_embedding 결과:", user_keyword_embedding_1[:2])
print("encode_sentence 결과:", user_keyword_embedding_2[:2])

# 비교
if (user_keyword_embedding_1 == user_keyword_embedding_2).all(): print("결과 같음.")
else: print("결과 다름.")


get_keyword_embedding 결과: [[ 1.65658027e-01 -1.13808244e-01  4.27066118e-01 -2.67661929e+00
  -1.99760988e-01  3.39747310e-01 -1.48910716e-01  2.71993279e-02
   5.58491647e-02 -5.22645831e-01 -9.37212110e-02 -6.74195290e-02
  -2.04050113e-04  1.12791799e-01  6.93494558e-01  1.81714833e-01
  -7.01291189e-02  1.43722698e-01  1.38260722e-01 -4.27141368e-01
  -1.14050925e-01 -1.45194024e-01  1.20403111e-01 -1.15253692e-02
  -1.71053186e-01 -3.44931811e-01 -1.46150932e-01 -3.20027173e-02
   2.75905788e-01  5.44158638e-01  5.82796633e-02 -2.29438737e-01
   2.47965306e-02 -4.18786854e-02  8.68668035e-02 -1.06079578e-01
   3.52121703e-02  5.16643189e-02  2.27658246e-02 -6.76672459e-01
   2.34706655e-01 -2.04829946e-01  2.47892514e-01  7.72284567e-02
   4.86931562e-01  1.58777852e-02 -4.55449045e-01  1.93926126e-01
   1.31212831e-01 -8.71576443e-02  5.62587241e-03 -3.29857282e-02
   4.11891311e-01 -1.05606364e-02  3.53515267e-01 -2.70156264e-01
   1.26030907e-01 -3.10156763e-01 -2.97540367e-01 

In [39]:
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드

# 같은 키워드로 여러 번 임베딩 생성
user_keyword_embedding_1 = encode_sentence(user_keyword)
user_keyword_embedding_2 = encode_sentence(user_keyword)
user_keyword_embedding_3 = encode_sentence(user_keyword)

# 각 임베딩 값 출력 (처음 두 개만)
print("첫 번째 임베딩 결과 (두 개):", user_keyword_embedding_1[:2])
print("두 번째 임베딩 결과 (두 개):", user_keyword_embedding_2[:2])
print("세 번째 임베딩 결과 (두 개):", user_keyword_embedding_3[:2])

# 비교
if (user_keyword_embedding_1 == user_keyword_embedding_2).all() and (user_keyword_embedding_1 == user_keyword_embedding_3).all():
    print("모든 임베딩 값이 같습니다.")
else:
    print("임베딩 값이 다릅니다.")


첫 번째 임베딩 결과 (두 개): [[ 2.08431646e-01 -8.82846043e-02  1.30654275e-01 -3.01224780e+00
  -2.01484397e-01 -3.12580504e-02 -1.55759946e-01  2.15345785e-01
  -1.16858518e+00 -1.87271312e-01 -1.65067136e-01 -1.55261680e-01
  -1.44063532e-02  9.34660658e-02  1.28356516e-01  2.63254732e-01
  -2.16740575e-02  1.28483325e-02 -1.43218994e-01 -1.18892491e-01
  -1.01618491e-01 -2.74763823e-01 -4.96719033e-02 -5.90935796e-02
   9.02667362e-03 -1.70134187e-01 -2.97845185e-01 -2.17945620e-01
   1.23348296e-01  1.49542972e-01  7.41464719e-02  5.83034158e-02
   1.26595600e-02  2.46960521e-02 -9.77789760e-02  9.13587213e-02
   1.80947706e-01 -6.11073077e-02 -2.23005235e-01 -1.51315778e-01
   2.78463751e-01 -3.74875873e-01  1.35557517e-01  4.70739417e-02
   6.16744161e-03 -2.16611907e-01 -8.32578018e-02  2.30610911e-02
   2.75467724e-01 -3.86728090e-03 -7.32254535e-02  1.91959679e-01
  -3.29504013e-02 -1.19356310e-03  5.19287288e-01  6.51217997e-04
  -2.17802510e-01 -6.22674562e-02 -7.87850544e-02  1.9717

In [43]:
import pandas as pd
import numpy as np

# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 매칭되는 벡터값 확인
def is_embedding_match(row_embedding):
    if row_embedding is not None:
        return np.all(np.array(user_keyword_embedding) == np.array(row_embedding))
    return False

# 매칭되는 행 찾기
matching_rows = df_inheritance[df_inheritance['키워드임베딩'].apply(is_embedding_match)]

if not matching_rows.empty:
    print("매칭되는 임베딩이 있는 행:")
    print(matching_rows)
else:
    print("매칭되는 임베딩이 없습니다.")


매칭되는 임베딩이 없습니다.


In [45]:
print(df_inheritance['키워드'].iloc[0])

['신청', '상속', '부동산', '주택', '사망', '건물', '계약', '회사', '재산', '호주']


In [48]:
# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)
first_row_embedding = df_inheritance['키워드임베딩'].iloc[0]

print("사용자 키워드 임베딩:", user_keyword_embedding)
print("첫 번째 행의 임베딩:", first_row_embedding)
# 임베딩 비교
if (user_keyword_embedding == first_row_embedding).all():  # NumPy 배열인 경우
    print("두 임베딩이 같습니다.")
else:
    print("두 임베딩이 다릅니다.")

사용자 키워드 임베딩: [[-1.33970901e-01 -3.99938315e-01 -9.70683768e-02 -3.12653661e+00
  -1.05491988e-01 -4.73101027e-02 -1.86071083e-01  2.04928115e-01
  -3.15773606e-01  1.58532724e-01 -4.85108979e-02 -1.18976735e-01
  -1.57061309e-01  2.61680316e-02 -5.42564690e-02  1.77757964e-01
   2.57029444e-01 -1.35125950e-01 -2.39797696e-01 -3.30991209e-01
  -4.39392895e-01 -2.05540180e-01  4.10773039e-01  4.52277184e-01
  -4.39014763e-01  2.30232641e-01 -3.00855964e-01 -3.27515632e-01
   2.04158053e-01  5.76157160e-02  2.94012517e-01  7.78875425e-02
   7.44750649e-02 -4.14165020e-01  1.01797031e-02 -2.65601665e-01
   3.58266592e-01  3.81460190e-01  1.71234369e-01  5.03078103e-02
  -8.86211768e-02  2.81853795e-01 -3.55571270e-01  3.67912501e-02
   3.09379399e-03  3.02261859e-01 -2.33066306e-01 -1.02756195e-01
   4.71889257e-01  4.43073601e-01  4.74642247e-01  4.15889591e-01
   5.44613488e-02 -3.20302635e-01  3.76506835e-01  5.39173298e-02
   4.95068640e-01 -1.17667854e-01 -4.97581989e-01  2.31855348e-

In [53]:
# 사용자 입력
user_keyword = "신청상속부동산주택사망건물계약회사재산호주"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)
first_row_embedding = df_inheritance['키워드임베딩'].iloc[0]

print("사용자 키워드 임베딩:", user_keyword_embedding)
print("첫 번째 행의 임베딩:", first_row_embedding)
# 임베딩 비교
if (user_keyword_embedding == first_row_embedding).all():  # NumPy 배열인 경우
    print("두 임베딩이 같습니다.")
else:
    print("두 임베딩이 다릅니다.")

사용자 키워드 임베딩: [[ 2.08431646e-01 -8.82846043e-02  1.30654275e-01 -3.01224780e+00
  -2.01484397e-01 -3.12580504e-02 -1.55759946e-01  2.15345785e-01
  -1.16858518e+00 -1.87271312e-01 -1.65067136e-01 -1.55261680e-01
  -1.44063532e-02  9.34660658e-02  1.28356516e-01  2.63254732e-01
  -2.16740575e-02  1.28483325e-02 -1.43218994e-01 -1.18892491e-01
  -1.01618491e-01 -2.74763823e-01 -4.96719033e-02 -5.90935796e-02
   9.02667362e-03 -1.70134187e-01 -2.97845185e-01 -2.17945620e-01
   1.23348296e-01  1.49542972e-01  7.41464719e-02  5.83034158e-02
   1.26595600e-02  2.46960521e-02 -9.77789760e-02  9.13587213e-02
   1.80947706e-01 -6.11073077e-02 -2.23005235e-01 -1.51315778e-01
   2.78463751e-01 -3.74875873e-01  1.35557517e-01  4.70739417e-02
   6.16744161e-03 -2.16611907e-01 -8.32578018e-02  2.30610911e-02
   2.75467724e-01 -3.86728090e-03 -7.32254535e-02  1.91959679e-01
  -3.29504013e-02 -1.19356310e-03  5.19287288e-01  6.51217997e-04
  -2.17802510e-01 -6.22674562e-02 -7.87850544e-02  1.97172403e-

In [49]:
# 사용자 키워드에 대한 임베딩을 찾기 위한 함수
def get_keyword_embedding(user_keyword, df):
    # 키워드가 포함된 행을 필터링
    matching_rows = df[df['키워드'].apply(lambda x: user_keyword in x if isinstance(x, list) else False)]
    
    if not matching_rows.empty:
        # 첫 번째 매칭된 키워드의 임베딩을 반환
        return matching_rows['키워드임베딩'].iloc[0]
    else:
        return None  # 매칭되는 키워드가 없으면 None 반환

# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding2 = get_keyword_embedding(user_keyword, df_inheritance)

print("사용자 키워드 임베딩:", user_keyword_embedding2)
print("첫 번째 행의 임베딩:", first_row_embedding)
# 임베딩 비교
if (user_keyword_embedding2 == first_row_embedding).all():  # NumPy 배열인 경우
    print("두 임베딩이 같습니다.")
else:
    print("두 임베딩이 다릅니다.")


사용자 키워드 임베딩: [[ 8.19379464e-02  5.26036806e-02  3.07946980e-01 -3.76139021e+00
   4.87393476e-02  3.01731616e-01 -1.18151486e-01  1.80137977e-01
  -2.17863858e-01 -2.01845288e-01 -1.54184818e-01  1.15717366e-01
   2.58666337e-01 -1.09485928e-02  5.22862732e-01  1.85482457e-01
   1.28475457e-01 -1.44635752e-01  3.22932750e-02 -2.56298780e-01
  -1.09786972e-01 -8.92716199e-02  2.19271109e-01 -6.56160861e-02
  -5.82302749e-01 -2.17919145e-02 -2.44034544e-01  2.69377287e-02
   1.54327795e-01  7.84756541e-01 -1.24786636e-02 -2.42831096e-01
   1.36727884e-01 -1.04116715e-01  1.98231697e-01 -1.37855217e-01
   8.37820098e-02  1.42861575e-01  3.14960539e-01 -4.43485171e-01
   7.53021659e-03 -2.78165769e-02  2.67103594e-03 -1.99171036e-01
   3.02799910e-01  1.25628203e-01 -3.98505539e-01  2.04539821e-01
   4.95789610e-02  1.22873873e-01 -2.34561175e-01  1.19103111e-01
   5.47493100e-01 -2.68747121e-01  4.95703161e-01 -3.83478701e-01
   2.18053862e-01 -2.67521143e-01 -1.81534365e-01  5.19793510e-

In [17]:
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)
print('test' ,user_keyword_embedding[:2])

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판례내용이유임베딩'] is not None:
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['판례내용이유임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 :")
for 사건번호, score in top_n_contents:
    사건명 = df_inheritance.loc[df_inheritance['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

test [[ 2.08431646e-01 -8.82846043e-02  1.30654275e-01 -3.01224780e+00
  -2.01484397e-01 -3.12580504e-02 -1.55759946e-01  2.15345785e-01
  -1.16858518e+00 -1.87271312e-01 -1.65067136e-01 -1.55261680e-01
  -1.44063532e-02  9.34660658e-02  1.28356516e-01  2.63254732e-01
  -2.16740575e-02  1.28483325e-02 -1.43218994e-01 -1.18892491e-01
  -1.01618491e-01 -2.74763823e-01 -4.96719033e-02 -5.90935796e-02
   9.02667362e-03 -1.70134187e-01 -2.97845185e-01 -2.17945620e-01
   1.23348296e-01  1.49542972e-01  7.41464719e-02  5.83034158e-02
   1.26595600e-02  2.46960521e-02 -9.77789760e-02  9.13587213e-02
   1.80947706e-01 -6.11073077e-02 -2.23005235e-01 -1.51315778e-01
   2.78463751e-01 -3.74875873e-01  1.35557517e-01  4.70739417e-02
   6.16744161e-03 -2.16611907e-01 -8.32578018e-02  2.30610911e-02
   2.75467724e-01 -3.86728090e-03 -7.32254535e-02  1.91959679e-01
  -3.29504013e-02 -1.19356310e-03  5.19287288e-01  6.51217997e-04
  -2.17802510e-01 -6.22674562e-02 -7.87850544e-02  1.97172403e-01
   3.